#### **Welcome to Assignment 3 on Deep Learning for Computer Vision.**
This assignment is based on the content you learned in Week-3.

#### **Instructions**
1. Use Python 3.x to run this notebook
2. Write your code only in between the lines 'YOUR CODE STARTS HERE' and 'YOUR CODE ENDS HERE'. You should not change anything else in the code cells, if you do, the answers you are supposed to get at the end of this assignment might be wrong.
3. Read documentation of each function carefully.
4. All the Best!

# Pytorch operations

You will perform basic elementwise operations between two matrices

Step1: rearrange the give input x into a tensor of shape (10,5,3) and store it in variable x_resize \\
Step2: Generate a gaussian random tensor (mean =0, variance=1) of shape (5,3) and store it in variable y1 \\
Step3:  Generate a uniform random tensor (interval [-10,10)) of shape (5,3) and store it in variable y2 \\
Step4: Perform elementwise multiplication between x_resize and y1, store it in variable mul_output1 \\
Step5: Find the max element along the last dimension of mul_output1 and store its value in final_output1 \\
Step6: Repeat Step4 and Step5 for x_resize and y2 and store the corresponding values in the variables, mul_output2 and final_output2.

In [1]:
import torch

In [2]:
#DO NOT CHANGE THE SEED
torch.manual_seed(2)

x = torch.randint(low=0,high=256,size=(1,150))
### YOUR CODE STARTS HERE ###

x_resize = x.view(10, 5, 3)

# Use torch.randn()
y1 = torch.randn(5, 3)

# Use torch.rand()
y2 = torch.rand(5, 3) * 20 - 10

mul_output1 = x_resize * y1

final_output1,_ = mul_output1.max(dim=2)

mul_output2 = x_resize * y2

final_output2,_ = mul_output2.max(dim=2)

### YOUR CODE ENDS HERE ###

print(torch.mean(final_output1+final_output2))

tensor(879.7391)


### Question 1. What is mean of the sum of final_output1 and final_output2? (Select the closest value)

a) 875

b) 880

c) 883

d) 870


# Custom loss function and its Gradients

In this we will implement a custom loss function and calculate the gradient of this loss function w.r.t two input tensors.

The loss function takes two inputs, y_true (given) and y_pred. Follow the below step to generate y_pred.

1) Calculate the matrix-vector multiplication between 'M' and 'v' and store it in the variable "y_pred".


In [3]:
import torch
import torch.nn as nn


#DO NOT CHANGE THE SEED
torch.manual_seed(0)

y_true = torch.tensor([0.3, 0.7, 0.6])

m, n = 3, 2
M = torch.randn(m, n, requires_grad=True)
v = torch.randn(n, requires_grad=True)

### YOUR CODE STARTS HERE ###
y_pred = torch.matmul(M, v)
### YOUR CODE ENDS HERE ###

Next, implement a custom loss function in PyTorch that computes the mean squared error (MSE) between two tensors (y_true, y_pred), but only for elements where the target tensor y_true is greater than 0.5. Save the calculated MSE loss in a variable named "loss".

*(Hint: First create a mask on y_true and then multiply the mask with the loss)*

In [5]:
import torch
import torch.nn as nn


#DO NOT CHANGE THE SEED
torch.manual_seed(0)

class CustomMSELoss(nn.Module):
    def __init__(self):
        super(CustomMSELoss, self).__init__()

    def forward(self, y_pred, y_true):
        ### YOUR CODE STARTS HERE ###
        mask = y_true > 0.5
        squared_diff = (y_pred - y_true) ** 2
        masked_squared_diff = squared_diff * mask

        # For calculating the mean, take the sum of masked_squared_diff and divide by 3.
        loss = masked_squared_diff.sum() / mask.sum().float()
        ### YOUR CODE ENDS HERE ###
        return loss

### DO NOT CHANGE ###
loss_fn = CustomMSELoss()
loss = loss_fn(y_pred, y_true)
print("Loss: ", loss)

Loss:  tensor(3.0486, grad_fn=<DivBackward0>)


Next, calculate the gradient of the loss w.r.t tensors M and v.

Store the gradient w.r.t M in a variable grad_M

Store the gradient w.r.t v in a variable grad_v

(Hint: Look into the autograd library of Pytorch. https://pytorch.org/docs/stable/generated/torch.autograd.grad.html)


In [6]:
### DO NOT CHANGE ###
# Compute gradients using autograd
loss.backward()  # This computes gradients for all tensors that have requires_grad=True

### YOUR CODE STARTS HERE ###
grad_M = M.grad
grad_v = v.grad
### YOUR CODE ENDS HERE ###

print("Sum of Gradient Means: ", grad_M.mean()+grad_v.mean())

Sum of Gradient Means:  tensor(2.9457)


### Question 2: What is the loss value calculated using the Custom Loss function? (Select the closest value)

a) 1

b) 2.5

c)1.5

d) 2



### Question 3: What is sum of the means of the gradients, grad_M and grad_v? (Select the closest value)

a) 2

b) 1.7

c) 2.5

d) 3



# Multi Layer Perceptron

In this, we will implement a simple multilayer feedforward neural network using PyTorch with the following specifications:

Input dimension: 10

Hidden layer 1 dimension: 20

Hidden layer 2 dimension:  50

Output dimension: 5

Activation function: ReLU for the hidden layer

Your function should take a tensor x of shape (batch_size, 10) as input and return the output of the neural network.

In [10]:
import torch
import torch.nn as nn

#DO NOT CHANGE THE SEED
torch.manual_seed(2)

# Define the multi-layer neural network class
class MultiLayerNN(nn.Module):
    def __init__(self):
        super(MultiLayerNN, self).__init__()

        ### YOUR CODE STARTS HERE ###
        self.fc1 = nn.Linear(10, 20) # Input layer to first hidden layer
        self.fc2 = nn.Linear(20, 50)  # First hidden layer to second hidden layer
        self.fc3 = nn.Linear(50, 5)   # Second hidden layer to output layer
        ### YOUR CODE ENDS HERE ###

    def forward(self, x):
       ### YOUR CODE STARTS HERE ###
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))

        ### YOUR CODE ENDS HERE ###
        x = self.fc3(x)
        return x

# DO NOT CHANGE:
model = MultiLayerNN()
input_tensor = torch.randn(32, 10)
output = model(input_tensor)
print("Mean of the output: ", output.mean() * 1000)

Mean of the output:  tensor(1.8321, grad_fn=<MulBackward0>)


### Question 4: What is mean of the output? (Select the closest value)

a) 1.6

b) 2.2

c) 3

d) 1.8



#Training of Multilayer Perceptron on a random dataset

In this we will use the Mulitlayer Perceptron defined in question 4 and train it on a random dataset.

In [11]:
# Please DO NOT modify this cell.

import os
import os.path as osp
import random

import numpy as np
import torch

def set_seed(seed: int):

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
    os.environ["PYTHONHASHSEED"] = str(seed)

In [12]:
# Please DO NOT modify this cell.

num_features = 10
classes = [0, 1, 2, 3, 4]
num_classes = len(classes)

num_samples = 100
num_train = 70
num_test = num_samples - num_train

In [13]:
# Please DO NOT modify this cell.
# We are creating a random feature set and a random label set.
# The features and labels have no semantic meaning and might as well be garbage.

set_seed(2022)

features = np.random.random_sample((num_samples, num_features))
labels = np.random.choice(classes, size = num_samples)

# Train-test split
x_train = features[:num_train]
x_test = features[num_train:num_samples]

x_train = torch.Tensor(x_train)
x_test = torch.Tensor(x_test)

y_train = labels[:num_train]
y_test = labels[num_train:num_samples]

y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [14]:
# Please DO NOT modify this cell.

print(f"Train features: {x_train.shape}")
print(f"Test features: {x_test.shape}")

print(f"Train labels: {y_train.shape}")
print(f"Train labels: {y_test.shape}")


Train features: torch.Size([70, 10])
Test features: torch.Size([30, 10])
Train labels: torch.Size([70])
Train labels: torch.Size([30])


### Create train and test TensorDatasets from the respective numpy arrays

In [15]:
# Create train and test TensorDatasets from the respective numpy arrays

#### YOUR CODE STARTS HERE ####
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(x_train, y_train)
test_dataset = TensorDataset(x_test, y_test)

#### YOUR CODE ENDS HERE ####

### Create dataloaders using the datasets created in the previous cell.

In [16]:
# Create dataloaders using the datasets created in the previous cell.
# Use a batch size of 64

#### YOUR CODE STARTS HERE ####

batch_size = 64

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

#### YOUR CODE ENDS HERE ####

### Training function

In [17]:
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    epoch_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):

        #### YOUR CODE STARTS HERE ####

        # send the data, target to the device
        data = data.to(device)
        target = target.to(device)

        # flush out the gradients stored in optimizer
        optimizer.zero_grad()

        # pass the batch to the model and assign the output to variable named y_pred
        output = model(data)

        # calculate the loss (use CrossEntropyLoss in pytorch)
        loss = criterion(output, target)

        # do a backward pass
        loss.backward()

        # update the weights
        optimizer.step()

        #### YOUR CODE ENDS HERE ####

        # Store loss
        epoch_loss += loss.item() * data.shape[0]

    print(f"Train Average Loss: {epoch_loss/len(train_loader.dataset):.2f}")

### Test function

In [18]:
def test(model, device, test_loader, criterion, mode):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            #### YOUR CODE STARTS HERE ####

            # send data, target to the device
            data = data.to(device)
            target = target.to(device)

            # pass the image to the model and assign the output to variable named output
            output = model(data)
            #### YOUR CODE ENDS HERE ####

            test_loss += criterion(output, target).item() * data.shape[0]  # sum up batch loss
            pred = output.argmax(dim = 1, keepdim = True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_acc = 100. * correct / len(test_loader.dataset)

    print(f"{mode} Average loss: {test_loss:.2f}")
    print(f"{mode} Accuracy: {correct}/{len(test_loader.dataset)} ({test_acc:.2f}%)")

In [20]:
set_seed(2022)

num_epochs = 300

#### YOUR CODE STARTS HERE ####

# check availability of GPU and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize MLP model
model = MultiLayerNN().to(device)

# Define Adam Optimizer with a learning rate of 0.001
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Define CrossEntropyLoss as the criterion
criterion = nn.CrossEntropyLoss()

#### YOUR CODE ENDS HERE ####

for epoch in range(1, num_epochs+1):
    print(f"\nEpoch: {epoch}/{num_epochs}")

    train(model, device, train_loader, optimizer, criterion, epoch)
    test(model, device, test_loader, criterion, mode = "Test")


Epoch: 1/300
Train Average Loss: 1.61
Test Average loss: 1.62
Test Accuracy: 6/30 (20.00%)

Epoch: 2/300
Train Average Loss: 1.61
Test Average loss: 1.62
Test Accuracy: 6/30 (20.00%)

Epoch: 3/300
Train Average Loss: 1.61
Test Average loss: 1.62
Test Accuracy: 6/30 (20.00%)

Epoch: 4/300
Train Average Loss: 1.61
Test Average loss: 1.62
Test Accuracy: 6/30 (20.00%)

Epoch: 5/300
Train Average Loss: 1.61
Test Average loss: 1.62
Test Accuracy: 6/30 (20.00%)

Epoch: 6/300
Train Average Loss: 1.61
Test Average loss: 1.62
Test Accuracy: 6/30 (20.00%)

Epoch: 7/300
Train Average Loss: 1.61
Test Average loss: 1.62
Test Accuracy: 6/30 (20.00%)

Epoch: 8/300
Train Average Loss: 1.61
Test Average loss: 1.62
Test Accuracy: 6/30 (20.00%)

Epoch: 9/300
Train Average Loss: 1.60
Test Average loss: 1.62
Test Accuracy: 6/30 (20.00%)

Epoch: 10/300
Train Average Loss: 1.60
Test Average loss: 1.62
Test Accuracy: 6/30 (20.00%)

Epoch: 11/300
Train Average Loss: 1.60
Test Average loss: 1.62
Test Accuracy: 

In [21]:
test(model, device, train_loader, criterion, mode = "Train")

Train Average loss: 1.22
Train Accuracy: 42/70 (60.00%)


### Question 5

Report the final train accuracy (If you are not getting the exact number shown in
options, please report the closest number).

a. 100%

b. 53%

c. 61%

d. 79%



In [22]:
test(model, device, test_loader, criterion, mode = "Test")

Test Average loss: 1.91
Test Accuracy: 5/30 (16.67%)


### Question 6

Report the final test accuracy (If you are not getting the exact number shown in
options, please report the closest number).

a. 30%

b. 40%

c. 15%

d. 25%

